# Aula 06 - Manipulação Intermediária de Dados

## **Resumo**: Exerícios de merge, alongamento e alargamento de DataFrames no Pandas

# I CARREGAMENTO DOS PACOTES NECESSÁRIOS

In [4]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path
from tqdm import tqdm

# II DIRETÓRIOS

In [5]:
os.chdir("G:\\Meu Drive\\Selection\\esm_selection")
root_dir   = Path(os.getcwd())
input_dir  = Path(root_dir, "Data/Input")
output_dir = Path(root_dir, "Data/Output")
figure_dir = Path(root_dir, 'Figures/')
output_dir

WindowsPath('G:/Meu Drive/Selection/esm_selection/Data/Output')

# III ABERTURA DOS DADOS

In [51]:
pagamentos = (pd.read_table(Path(input_dir, "Folha_pagamento_comma.txt"), sep = "," )
              .drop(["Nome", "Salário", "Encargos"], axis = 1)
              .assign(salario = lambda df: np.random.triangular(1320, 2500, 49000, size = df.shape[0]),
                      encargos = lambda df: 0.15*df["salario"] + np.random.triangular(100, 500, 2000, size = df.shape[0])
                                            +  np.random.triangular(10, 50, 100, size = df.shape[0])**2,
                      custo_mensal = lambda df: df["encargos"] + df["salario"],
                      custo_anual = lambda df: df["custo_mensal"]*12
                      )
              .rename(columns = {"CPF_fake": "cpf",
                                 "salario": "salario_2022",
                                 "encargos": "encargos_2022",
                                 "custo_mensal": "custo_mensal_2022",
                                 "custo_anual": "custo_anual_2022"} )
              .assign(salario_2023 = lambda df: 1.20*df["salario_2022"],
                      encargos_2023  = lambda df: 1.20*df["encargos_2022"],
                      custo_mensal_2023  = lambda df: 1.20*df["custo_mensal_2022"],
                      custo_anual_2023  = lambda df: 1.20*df["custo_anual_2022"]
                      )
              )

pagamentos

,cpf,salario_2022,encargos_2022,custo_mensal_2022,custo_anual_2022,salario_2023,encargos_2023,custo_mensal_2023,custo_anual_2023
0,908777362057,7770.572573,5797.353444,13567.926017,162815.112204,9324.687088,6956.824132,16281.511220,195378.134645
1,324268288911,14471.245893,4782.346582,19253.592475,231043.109703,17365.495071,5738.815899,23104.310970,277251.731643
2,766148980149,4041.839442,2675.026754,6716.866196,80602.394348,4850.207330,3210.032104,8060.239435,96722.873218
3,575223597306,15934.277988,5200.940121,21135.218109,253622.617309,19121.133586,6241.128145,25362.261731,304347.140771
4,900619489281,3734.132159,4491.450285,8225.582443,98706.989320,4480.958590,5389.740342,9870.698932,118448.387184
...,...,...,...,...,...,...,...,...,...
7958,987599552888,29737.956709,7036.353232,36774.309940,441291.719283,35685.548050,8443.623878,44129.171928,529550.063140
7959,577833528199,32065.018319,7637.138842,39702.157161,476425.885928,38478.021983,9164.566610,47642.588593,571711.063114
7960,920324446993,8149.878535,2805.566032,10955.444567,131465.334809,9779.854242,3366.679239,13146.533481,157758.401770
7961,631234133516,10490.035108,5548.140741,16038.175849,192458.110188,12588.042130,6657.768889,19245.811019,230949.732226


In [52]:
pessoais = (pagamentos
            .filter(["cpf"], axis = 1)
            .assign(idade = lambda df:  np.random.triangular(14, 35, 65, df.shape[0]).round(0),
                    sexo = lambda df:  np.random.binomial(1, 0.75, size = df.shape[0])
                    )
            .astype({"idade": "int"})
            .sample(frac = 0.70)
            .rename(columns = {"cpf": "CPF_fake"})
            .melt(id_vars = ["CPF_fake"], value_vars = ["idade", "sexo"], var_name = "pessoais", value_name = "values")
            )

pessoais

,CPF_fake,pessoais,values
0,486042470594,idade,38
1,552816674064,idade,45
2,353936918775,idade,48
3,569947284680,idade,31
4,664893677867,idade,55
...,...,...,...
11143,194277189012,sexo,1
11144,839819481013,sexo,1
11145,483411370264,sexo,1
11146,178957098288,sexo,1


# Exercício 1: Merge

In [53]:
# a)
data = (pagamentos
        .merge(pessoais, how = "left", left_on = "cpf", right_on = "CPF_fake")
        )

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13537 entries, 0 to 13536
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   cpf                13537 non-null  int64  
 1   salario_2022       13537 non-null  float64
 2   encargos_2022      13537 non-null  float64
 3   custo_mensal_2022  13537 non-null  float64
 4   custo_anual_2022   13537 non-null  float64
 5   salario_2023       13537 non-null  float64
 6   encargos_2023      13537 non-null  float64
 7   custo_mensal_2023  13537 non-null  float64
 8   custo_anual_2023   13537 non-null  float64
 9   CPF_fake           11148 non-null  float64
 10  pessoais           11148 non-null  object 
 11  values             11148 non-null  float64
dtypes: float64(10), int64(1), object(1)
memory usage: 1.3+ MB


In [54]:
# b)
data = (pagamentos
        .merge(pessoais, how = "right", left_on = "cpf", right_on = "CPF_fake")
        )

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11148 entries, 0 to 11147
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   cpf                11148 non-null  int64  
 1   salario_2022       11148 non-null  float64
 2   encargos_2022      11148 non-null  float64
 3   custo_mensal_2022  11148 non-null  float64
 4   custo_anual_2022   11148 non-null  float64
 5   salario_2023       11148 non-null  float64
 6   encargos_2023      11148 non-null  float64
 7   custo_mensal_2023  11148 non-null  float64
 8   custo_anual_2023   11148 non-null  float64
 9   CPF_fake           11148 non-null  int64  
 10  pessoais           11148 non-null  object 
 11  values             11148 non-null  int32  
dtypes: float64(8), int32(1), int64(2), object(1)
memory usage: 1.1+ MB


In [55]:
# c)
data = (pagamentos
        .merge(pessoais, how = "inner", left_on = "cpf", right_on = "CPF_fake")
        )

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11148 entries, 0 to 11147
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   cpf                11148 non-null  int64  
 1   salario_2022       11148 non-null  float64
 2   encargos_2022      11148 non-null  float64
 3   custo_mensal_2022  11148 non-null  float64
 4   custo_anual_2022   11148 non-null  float64
 5   salario_2023       11148 non-null  float64
 6   encargos_2023      11148 non-null  float64
 7   custo_mensal_2023  11148 non-null  float64
 8   custo_anual_2023   11148 non-null  float64
 9   CPF_fake           11148 non-null  int64  
 10  pessoais           11148 non-null  object 
 11  values             11148 non-null  int32  
dtypes: float64(8), int32(1), int64(2), object(1)
memory usage: 1.1+ MB


In [62]:
# d)
data = (pagamentos
        .merge(pessoais, how = "outer", left_on = "cpf", right_on = "CPF_fake")
        )

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13537 entries, 0 to 13536
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   cpf                13537 non-null  int64  
 1   salario_2022       13537 non-null  float64
 2   encargos_2022      13537 non-null  float64
 3   custo_mensal_2022  13537 non-null  float64
 4   custo_anual_2022   13537 non-null  float64
 5   salario_2023       13537 non-null  float64
 6   encargos_2023      13537 non-null  float64
 7   custo_mensal_2023  13537 non-null  float64
 8   custo_anual_2023   13537 non-null  float64
 9   CPF_fake           11148 non-null  float64
 10  pessoais           11148 non-null  object 
 11  values             11148 non-null  float64
dtypes: float64(10), int64(1), object(1)
memory usage: 1.3+ MB


# Exercíco 2: Alargamento e Alongamento

In [129]:
# a)

data_w = (data
          .melt(id_vars = ["cpf", "pessoais", "values",
                           "custo_mensal_2022", "custo_mensal_2023",
                           "custo_anual_2022", "custo_anual_2023"],
                value_vars = ["salario_2022", "salario_2023"],
                var_name = "ano_salario",
                value_name    = "salario"
               )
          .assign(ano_salario = lambda df: df["ano_salario"].str.slice(8,12).astype("int") )
          .melt(id_vars = ["cpf", "pessoais", "values",  "salario", "ano_salario",
                           "custo_anual_2022", "custo_anual_2023"],
                value_vars = ["custo_mensal_2022", "custo_mensal_2023"],
                var_name = "ano_mensal",
                value_name    = "custo_mensal"
                )
          .assign(ano_mensal = lambda df: df["ano_mensal"].str.slice(13,17).astype("int") )
          .melt(id_vars = ["cpf", "pessoais", "values",  "salario", "ano_salario", "ano_mensal",
                           "custo_mensal"],
                value_vars = ["custo_anual_2022", "custo_anual_2023"],
                var_name = "ano_anual",
                value_name    = "custo_anual"
                )
          .assign(ano_anual = lambda df: df["ano_anual"].str.slice(12,17).astype("int") )
          .drop_duplicates(keep = False)
          .query(" ano_salario == ano_mensal  & ano_salario == ano_anual")
          .rename(columns = {"ano_mensal": "ano"})
          .drop(["ano_anual", "ano_salario"], axis = 1)
          )
data_w

,cpf,pessoais,values,salario,ano,custo_mensal,custo_anual
0,908777362057,idade,23.0,7770.572573,2022,13567.926017,162815.112204
1,908777362057,sexo,1.0,7770.572573,2022,13567.926017,162815.112204
2,324268288911,NaN,NaN,14471.245893,2022,19253.592475,231043.109703
3,766148980149,idade,34.0,4041.839442,2022,6716.866196,80602.394348
4,766148980149,sexo,0.0,4041.839442,2022,6716.866196,80602.394348
...,...,...,...,...,...,...,...
108291,920324446993,idade,37.0,9779.854242,2023,13146.533481,157758.401770
108292,920324446993,sexo,1.0,9779.854242,2023,13146.533481,157758.401770
108293,631234133516,NaN,NaN,12588.042130,2023,19245.811019,230949.732226
108294,177894774588,idade,27.0,12800.437866,2023,18175.456702,218105.480424


In [130]:
# b)
data_wl = (data_w
           .pivot(index = ["cpf", "salario", "ano", "custo_mensal", "custo_anual"],
                  columns = ["pessoais"],
                  values = "values"
                  )
           .reset_index()
           .rename_axis(None, axis = 1)
           )

data_wl

,cpf,salario,ano,custo_mensal,custo_anual,NaN,idade,sexo
0,111114603517,24279.619866,2022,36552.720097,438632.641162,NaN,NaN,NaN
1,111114603517,29135.543840,2023,43863.264116,526359.169394,NaN,NaN,NaN
2,111264146353,39256.718730,2022,47512.769628,570153.235536,NaN,NaN,NaN
3,111264146353,47108.062476,2023,57015.323554,684183.882643,NaN,NaN,NaN
4,111674502938,14313.409258,2022,19755.811853,237069.742234,NaN,35.0,0.0
...,...,...,...,...,...,...,...,...
15921,999802902460,25371.922811,2023,32705.940163,392471.281962,NaN,42.0,1.0
15922,999939556661,10234.133114,2022,15622.013201,187464.158415,NaN,32.0,1.0
15923,999939556661,12280.959737,2023,18746.415841,224956.990098,NaN,32.0,1.0
15924,999990680210,26804.278045,2022,35241.851436,422902.217232,NaN,62.0,1.0
